# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Retrieve your trained model from the Model Registry.
3. Load batch data.
4. Predict batch data.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import datetime
import pandas as pd

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Model Registry </span>

In [ ]:
import hopsworks

project = hopsworks.login()

# Retrieve the model registry
mr = project.get_model_registry()

## <span style="color:#ff5f27;">🪝 Retrieve model from Model Registry</span>

In [ ]:
# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Load the XGBoost regressor model and label encoder from the saved model directory
xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>

In [ ]:
# Retrieve the 'air_quality_fv' feature view
feature_view = retrieved_model.get_feature_view()
feature_view.name, feature_view.version

## <span style="color:#ff5f27;">✨ Load Batch Data of last days</span>

First, you will need to fetch the training dataset that you created in the previous notebook.

In [ ]:
# Get the current date
today = datetime.date.today()

# Calculate a date threshold 30 days ago from the current date
date_threshold = today - datetime.timedelta(days=30)

# Convert the date threshold to a string format
str(date_threshold)

In [ ]:
# Retrieve batch data from the feature view with a start time set to the date threshold
batch_data = feature_view.get_batch_data(
    start_time=date_threshold,
)

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
# Transform the 'city_name' column in the batch data using the retrieved label encoder
encoded = encoder.transform(batch_data['city_name'])

# Concatenate the label-encoded 'city_name' with the original batch data
X_batch = pd.concat([batch_data, pd.DataFrame(encoded)], axis=1)

# Drop unnecessary columns ('date', 'city_name', 'unix_time') from the batch data
X_batch = X_batch.drop(columns=['date', 'city_name', 'unix_time'])

# Rename the newly added column with label-encoded city names to 'city_name_encoded'
X_batch = X_batch.rename(columns={0: 'city_name_encoded'})

# Extract the target variable 'pm2_5' from the batch data
y_batch = X_batch.pop('pm2_5')

X_batch.head(3)

In [ ]:
# Make predictions on the batch data using the retrieved XGBoost regressor model
predictions = xgboost_model.predict(X_batch)

# Display the first 5 predictions
predictions[:5]

---
## <span style="color:#ff5f27;">👾 Now try out the Streamlit App!</span>

In [ ]:
# !python3 -m streamlit run streamlit_app.py

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Air Quality tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai